# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

<div style="font-size: 20px"><b>Откроем датасет и изучим общую информацию о данных</b></div> <br>
Некоторые проблемы можно увидеть и обычным просмотром датасета через Excel, но воспользуемся некоторыми методами Pandas.

**Подключение библиотек**

In [ ]:
import pandas as pd

from pymystem3 import Mystem
m = Mystem()
from collections import Counter

**Для просмотра выведем содержимое датасета**

In [ ]:
data = pd.read_csv('/datasets/data.csv')
#print(data.head(15))
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Посмотрим общую информацию о датасете**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**: В столбцах 'days_employed' и 'total_income' присутствуют значения NaN, а также необходимо сметнить типы данных с float на int   

**Посмотрим уникальные значения и их количетсво в столбце** <i>'количество детей в семье'</i>

In [ ]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


**Вывод**: Подозрительное значение -1 и 20 

**Посмотрим уникальные значения и их количетсво в столбце** <i>'возраст клиента в годах'</i>

In [ ]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


**Вывод:** Возраст некоторых клиентов 0

**Посмотрим уникальные значения и их количетсво в столбце** <i>'уровень образования клиента'</i>

In [ ]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


**Вывод:** Одинаковые названия, разные регистры

**Посмотрим уникальные значения и их количетсво в столбце** <i>'пол клиента'</i>

In [ ]:
print(data['gender'].value_counts())
#print(data[data['gender'] == 'XNA'])

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


**Вывод:** Не понятное значение 'XNA'

**Посмотрим уникальные значения и их количетсво в столбце** <i>'цель получения кредита'</i>

In [ ]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод:** Посмотрев хотя бы на первые 3 записи уже деалем вывод, что необходимо произвести лемматизацию

**Посмотрим количество пропусков в датасете**

In [ ]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Вывод:** Как и говорилось, присутсвуют строки с NaN в двух столбцах

**Посмотрим количество дубликатов в датасете**

In [ ]:
print(data.duplicated().sum())

54


**Вывод:** Итого 54 дубликата

**Вывод**

Выявленны следующие проблемы, которые необходимо решить:
1. В столбцах 'общий трудовой стаж в днях' и 'ежемесячный доход' некорректный тип данных(тип с плавающей точкой);
2. В столбце 'количество детей в семье' встречаются как отрицательные значения(-1), так из начения, которые равны 20;
3. В столбце 'общий трудовой стаж в днях' встречаются отрицательные значения;
4. В столбцах 'общий трудовой стаж в днях' и 'ежемесячный доход' есть пустые строки;
5. Разный регистр букв в столбцах 'уровень образования клиента' и 'семейное положение';
6. В столбце 'возраст клиента в годах' встречаются значения 0;
7. В файле с данными встречаются дубликаты;
8. В одной записи в столбце 'пол клиента' не определен пол;
9. В столбце 'цель получения кредита' обнаружены одинаковые цели на кредит, но имеют разные названия;

## Шаг 2. Предобработка данных

### Обработка пропусков

**Так как нужно оценить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, а не от стажа работы или дохода, то заменим пропуски в столбце стаж работы на 0, а доход заменим на среднее значение по группам.<br> 
Обосновать данные пропуски можно следующими предположениями:**
1. Неофициальные работа или доход(заработок от акций, сдача недвижимости)
2. Либо были какие-либо ошибки при выгрузки из системы
3. Возможно, чтобы не указывать данную информацию, кредит был выдан под залог(недвижимости, машины). 


In [ ]:
# <замена пустых значений на 0 в столбце 'days_employed'>
data['days_employed'] = data['days_employed'].fillna(0)

In [ ]:
# <подсчет среднего значения столбца 'total_income' по группам столбца 'income_type'>
mean = data.groupby('income_type')['total_income'].mean()
print(mean)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64


In [ ]:
# <замена пустых значений столбца total_income на среднее по группам столбца income_type>
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [ ]:
# <проверяем, что избавились от пустых значений>
print(data.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод:** В данном случае произвели замену в столбце <i>days_employed</i>, приняли решение заменить стаж работы на 0. В пустых значениях столбца <i>total_income</i> решили заменить на среднее по группам столбца income_type.

### Замена типа данных

**Рассмотрим столбец** 'days_employed' **— общий трудовой стаж в днях. Заметим, что данные, которые мы получили являются артефактами(встречаются отрицательные значения).**

In [ ]:
#<уберем отрицательные значения, используя метод abs() и проверим>
data['days_employed'] = data['days_employed'].abs()
print(data['days_employed'])

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


**Представим информацию в столбце** 'days_employed' **следующим образом:**<br>
Найдем методом max() максимальный трудовой стаж и узнаем сколько этому человеку лет

In [ ]:
max_day = data['days_employed'].max()
print(max_day)

401755.40047533


In [ ]:
max_dob_maxday = data[data['days_employed'] == data['days_employed'].max()]['dob_years']
print(max_dob_maxday)

6954    56
Name: dob_years, dtype: int64


Известно, что кредиты выдаются людям, которым не больше 75 лет.<br> 
Трудовой стаж начисляется с 16 лет.<br>
Из тех данных, которые мы имеем, делаем вывод, что самый большой стаж у человека, которому 56 лет.<br>
Предположим, что этот человек нарабатывал стаж с 16 лет, т.е. стаж = 56-16 = 40 лет. Предлагаю посчитать индекс, с помощью которого переведем данные о стаже в более реальный вид.

In [ ]:
#<взяли, не учитывая високосные года>
index = ((max_dob_maxday - 16) * 365) / max_day
print(index)

6954    0.036341
Name: dob_years, dtype: float64


In [ ]:
# <создадим столбец days_employed_correct, чтобы в дальнейшем сравнить исходные данные с корректированными>
data['days_employed_correct'] = data['days_employed']
print(display(data))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_correct
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266.072047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,4529.316663
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,343937.404131
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,2113.346888
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,3112.481705


None


In [ ]:
#<коэффициент возьмем с точностью до 3 знаков>
data.loc[:,'days_employed_correct'] *= 0.036

In [ ]:
#<проверим появились ли более реальные значения>
#print(data.head(10))
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_correct
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,303.756229
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,144.892935
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,202.443214
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,148.490899
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,12249.578594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,163.055400
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,12381.746549
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,76.080488
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,112.049341


**Вывод:** Создали дополнительный столбец с открорректируемым доходом. Теперь мы наблюдаем более реальные значения, но стоит заметить, что это решение предложено для данного датасета.(теперь нет ни у кого стажа по 900 лет). В дальнейшем, сравним результаты.

**Приведем дни стажа к типу 'int', так как стаж определяется полными днями.**

In [ ]:
#<приводим столбец 'days_employed' к типу int>
data['days_employed'] = data['days_employed'].astype('int')

**Ежемесячный доход тоже приведем к типу 'int', как правило при выдаче кредита округляют сумму вашего дохода.**

In [ ]:
#<приводим столбец 'total_income' к типу int>
data['total_income'] = data['total_income'].astype('int')

In [ ]:
#<приводим столбец ''days_employed_correct' к типу int>
data['days_employed_correct'] = data['days_employed_correct'].astype('int')

In [ ]:
#<проверяем, что метод сработал>
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children                 21525 non-null int64
days_employed            21525 non-null int64
dob_years                21525 non-null int64
education                21525 non-null object
education_id             21525 non-null int64
family_status            21525 non-null object
family_status_id         21525 non-null int64
gender                   21525 non-null object
income_type              21525 non-null object
debt                     21525 non-null int64
total_income             21525 non-null int64
purpose                  21525 non-null object
days_employed_correct    21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


**Вывод**
1. Привели столбцы <i>'total_income'</i>, <i>'days_employed'</i> и <i>'days_employed_correct'</i> к типу int
2. Преобразовали значения столбца <i>'days_employed'</i> в более реалистичный вид и записали в столбец <i>'days_employed_correct'</i>

### Обработка дубликатов

In [ ]:
#<посчитаем сумму дубликатов>
print(data.duplicated().sum())

54


In [ ]:
#<методом drop_duplicates() удалим дубликаты и обновим индексы методом reset_index()>
data = data.drop_duplicates().reset_index(drop = True)

In [ ]:
#<проверка>
print(data.duplicated().sum())

0


In [ ]:
#<проверим уникальные значения столбца 'education'>
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
#<приведем все названия к одному регистру методом str.lower()>
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

In [ ]:
#<проверка>
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [ ]:
#<проверка>
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [ ]:
#<проверим не появились ли еще дубликаты>
print(data.duplicated().sum())

17


In [ ]:
#<методом drop_duplicates() удалим дубликаты и обновим индексы методом reset_index()>
data = data.drop_duplicates().reset_index(drop = True)

In [ ]:
#<проверим на этот раз>
print(data.duplicated().sum())

0


**Приведем к нормальным значениям столбец** 'children' **а именно, что делать со значениями -1 и 20**

In [ ]:
#<посмотрим уникальные значения и их количетсво в столбце 'children'>
print(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


**Так как строк с непонятным количеством детей всего 123 и это меньше 1% от всех записей, предлагаю удалить их.**

In [ ]:
#<строки где значение -1, заменим на 1>
data.loc[data['children'] == -1, 'children'] = 1

In [ ]:
#<строки где значение 20, заменим на 2>
data.loc[data['children'] == 20, 'children'] = 2

In [ ]:
print(data['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


In [ ]:
#<проверка>
print(data['dob_years'].value_counts())

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


**Вывод**
1. Стало понятно, что лучше сначала привести все к одному регистру(**.str.lower()**), а потом проверять на дубликаты(**duplicated().sum()**), иначе получится как у меня.
2. Возможные проблемы возникновения дубликатов: либо ошибки в работе системы(сбои во время ввода данных), либо человеческий фактор при вводе информации(а если касается заполнении заявки и отправки ее, то возможно человек дважды нажал кнопку отправить заявку и система приняла 2 одинаковые заявки)

### Лемматизация

In [ ]:
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [ ]:
#<получим массив со всеми целями на кредит>
purpose_massive = data['purpose'].unique()
print(purpose_massive)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [ ]:
#<склеим>
row = ', '.join(purpose_massive)
print(row)

покупка жилья, приобретение автомобиля, дополнительное образование, сыграть свадьбу, операции с жильем, образование, на проведение свадьбы, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, на покупку подержанного автомобиля, на покупку своего автомобиля, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, автомобили, заняться образованием, сделка с подержанным автомобилем, получение образования, автомобиль, свадьба, получение дополнительного образования, покупка своего жилья, операции с недвижимостью, получение высшего образования, свой автомобиль, сделка с автомобилем, профильное образование, высшее образование, покупка жилья для сдачи, на покупку автомобиля, ремонт жилью, заняться высшим образованием


In [ ]:
#<cтрочку row лемматизируем, получаем список лемм и посчитаем сколько раз употребляемтся слово> 
lemmas = m.lemmatize(row)
print(Counter(lemmas))

Counter({' ': 59, ', ': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод**
1. Выделяем следующие цели, на которые берут кредит: <b>недвижимость, автомобиль, образование, свадьба.</b>
2. При помощи функции **m.lemmatize()** лемматизации и **Counter()** определили какое количество раз употребляется то или иное слово

### Категоризация данных

**Классифицируем по количеству детей** <i>'children'</i>

In [ ]:
#<напишем функцию, которая классифицирует людей по количеству детей>
def children_group(children):
    if children == 0:
        return 'детей нет'
    if ( 0 < children <= 2):
        return 'немногодетная семья'
    return 'многодетная семья'
#print(children_group(0))
#print(children_group(2))
#print(children_group(3))

**Классифицируем по типу** <i>'family_status'</i>

In [ ]:
#<для отработки способа классифицируем то типу>
family_dict = data[['family_status_id', 'family_status']]
#<удалим дубликаты>
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
print(family_dict.head())

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  не женат / не замужем


In [ ]:
family_log = data[['family_status_id', 'debt']]
print(family_log)

       family_status_id  debt
0                     0     0
1                     0     0
2                     0     0
3                     0     0
4                     1     0
...                 ...   ...
21449                 1     0
21450                 0     0
21451                 1     1
21452                 0     1
21453                 0     0

[21454 rows x 2 columns]


**Классифицируем по уровню дохода** <i>'total_income'</i>

In [ ]:
#<посмотрим как можно классифицировать>
pd.options.display.float_format = '{:.3f}'.format
print(data['total_income'].max())
print(data['total_income'].min())
print(data['total_income'].mean())
print(data['total_income'].describe())

2265604
20667
167431.58371399273
count     21454.000
mean     167431.584
std       98060.605
min       20667.000
25%      107623.000
50%      151887.000
75%      202417.000
max     2265604.000
Name: total_income, dtype: float64


In [ ]:
#<напишем функцию, которая классифицирует людей по доходу>
def income_group(total_income):
    if total_income <= 107623:
        return 'низкий доход'
    if (total_income >= 107624) & (total_income <= 151887):
        return 'средний доход'
    return 'высокий доход'

In [ ]:
print(data['days_employed'].describe())

count    21454.000
mean     60355.051
std     133531.413
min          0.000
25%        622.000
50%       1821.500
75%       4797.000
max     401755.000
Name: days_employed, dtype: float64


In [ ]:
print(data['days_employed_correct'].describe())

count   21454.000
mean     2172.345
std      4807.110
min         0.000
25%        22.000
50%        65.000
75%       172.000
max     14463.000
Name: days_employed_correct, dtype: float64


In [ ]:
#<напишем функцию, которая классифицирует людей по стажу, используя столбец 'days_employed'>
def days_employed_group(days_employed):
    if days_employed <= 622:
        return 'маленький стаж'
    if (days_employed >= 623) & (days_employed <= 1821.5):
        return 'средний средний'
    return 'большой стаж'

In [ ]:
#<напишем функцию, которая классифицирует людей по стажу 'days_employed_correct'>
def days_employed_correct_group(days_employed_correct):
    if days_employed_correct <= 22:
        return 'маленький стаж'
    if (days_employed_correct >= 23) & (days_employed_correct <= 65):
        return 'средний средний'
    return 'большой стаж'

In [ ]:
#<создадим столбец>
data['days_employed_group'] = data['days_employed'].apply(days_employed_group)
data['days_employed_correct_group'] = data['days_employed_correct'].apply(days_employed_correct_group)

In [ ]:
#<посчитаем количество людей по каждому уровню стажа>
print(data['days_employed_group'].value_counts())

большой стаж       10727
маленький стаж      5367
средний средний     5360
Name: days_employed_group, dtype: int64


In [ ]:
print(data['days_employed_correct_group'].value_counts())

большой стаж       10684
маленький стаж      5450
средний средний     5320
Name: days_employed_correct_group, dtype: int64


In [ ]:
#<создадим столбец и проверим>
data['income_group'] = data['total_income'].apply(income_group)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_correct,days_employed_group,days_employed_correct_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,303,большой стаж,большой стаж,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,144,большой стаж,большой стаж,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,202,большой стаж,большой стаж,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,148,большой стаж,большой стаж,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,12249,большой стаж,большой стаж,высокий доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,163,большой стаж,большой стаж,высокий доход
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,12381,большой стаж,большой стаж,высокий доход
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,76,большой стаж,большой стаж,низкий доход
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,112,большой стаж,большой стаж,высокий доход


In [ ]:
#<посчитаем количество людей по каждому уровню дохода>
print(data['income_group'].value_counts())

высокий доход    10727
низкий доход      5364
средний доход     5363
Name: income_group, dtype: int64


**Классифицируем по целям получения кредита** <i>'purpose'</i>

In [ ]:
#<напишем функцию, которая классифицирует по целям кредита>
def purpose_group(purpose):
    if 'свад' in purpose:
             return 'свадьба'
    if 'авто' in purpose:
             return 'автомобиль'
    if 'образован' in purpose:
             return 'образование'
    if 'недвиж' or 'жил' in purpose:
             return 'недвижимость'
    return 'Ошибка'

In [ ]:
#<создадим столбец и проверим>
data['purpose_group'] = data['purpose'].apply(purpose_group)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_correct,days_employed_group,days_employed_correct_group,income_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,303,большой стаж,большой стаж,высокий доход,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,144,большой стаж,большой стаж,средний доход,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,202,большой стаж,большой стаж,средний доход,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,148,большой стаж,большой стаж,высокий доход,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,12249,большой стаж,большой стаж,высокий доход,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,163,большой стаж,большой стаж,высокий доход,недвижимость
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,12381,большой стаж,большой стаж,высокий доход,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,76,большой стаж,большой стаж,низкий доход,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,112,большой стаж,большой стаж,высокий доход,автомобиль


In [ ]:
#<посчитаем количество людей по каждоq цели кредита>
print(data['purpose_group'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64


**Вывод** 
1. Классифицировали данные различными способами, которые нам известны на данном этапе.
2. Классификацию проводили на основе требований, чтобы в дальнейшем использовать ее и ответить на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#<создадим сводную таблицу и посчитаем долю должников>
data_pivot = data.pivot_table(index = ['children'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_pivot['ratio'] = data_pivot[1] / data_pivot[0] * 100
print(data_pivot.sort_values(by = 'ratio', ascending = True))

debt             0        1  ratio
children                          
0        13028.000 1063.000  8.159
3          303.000   27.000  8.911
1         4410.000  445.000 10.091
2         1926.000  202.000 10.488
4           37.000    4.000 10.811
5            9.000      nan    nan


**Вывод**  Из всех, лучше всего закрывают кредиты без детей. У тех, у кого 3 ребенка также показатель ниже по сравнению с другими.(Интересно задаться вопросом: Возможно у кого 3 ребенка, их доля чаще использует какую-либо Господдержу?!). В значении с 5 детьми, видим NaN, возможно из-за деления 0/0.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#<посчитаем сумму должников по каждому статусу>
family_status_sum = family_log.groupby('family_status_id')['debt'].sum()
print(family_status_sum)

family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64


In [ ]:
#<посчитаем количество людей по каждому статусу>
family_status_count = family_log.groupby('family_status_id')['debt'].count()
print(family_status_count)

family_status_id
0    12339
1     4151
2      959
3     1195
4     2810
Name: debt, dtype: int64


In [ ]:
#<посчитаем долю людей, которые задолжали по кредиту>
family_dict['ratio'] = family_status_sum / family_status_count * 100
print(family_dict.sort_values(by = 'ratio', ascending = True))

   family_status_id          family_status  ratio
2                 2         вдовец / вдова  6.569
3                 3              в разводе  7.113
0                 0        женат / замужем  7.545
1                 1       гражданский брак  9.347
4                 4  не женат / не замужем  9.751


**Вывод** Стоит заметить, что самые большие показатели должников у <i>не женат / не замужем</i> и <i>гражданский брак</i>.
В первом случае понятно, что одному платить кредит намного сложнее. Во втором, можно объяснить тем, что в гражданском браке(если это рассматривать больше как сожительство), как правило, кредит выплачивает один человек, не обременяя другого. У остальных категорий с маленькими показателями, дела идут лучше. Находясь в их категории, человек, берущий кредит, чаще располагает более выгодным положением, при оформлении кредита(<i>женат / замужем</i> - работают вдвоем, есть подстраховка; <i>вдовец / вдова</i> - наследство от мужа/жены; <i>в разводе</i> - раздел имущества)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#<создадим сводную таблицу и посчитаем долю должников>
data_pivot_income = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'education', aggfunc = 'count')
data_pivot_income['ratio'] = data_pivot_income[1] / data_pivot_income[0] * 100
print(data_pivot_income.sort_values(by = 'ratio', ascending = True))

debt              0    1  ratio
income_group                   
высокий доход  9888  839  8.485
низкий доход   4937  427  8.649
средний доход  4888  475  9.718


**Вывод**
1. Самый низкий показатель должников у тех, у кого **низкий доход**. Обусловленно это скорее всего тем, что они более ответственно подходят к оформлению и выплате кредитов(стараются погасить раньше). 
2. Самый высоки показатель должников у тех, у кого **средний доход**. Возможно эта доля людей берут слишком дорогие кредиты, переоценивают свои силы в его выплате. А также возможно, берут несколько кредитов(недвижимость, автомобиль).  

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#<создадим сводную таблицу и посчитаем долю должников>
data_pivot_purpose = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_pivot_purpose['ratio'] = data_pivot_purpose[1] / data_pivot_purpose[0] * 100
print(data_pivot_purpose.sort_values(by = 'ratio', ascending = True))

debt               0    1  ratio
purpose_group                   
недвижимость   10029  782  7.797
свадьба         2138  186  8.700
образование     3643  370 10.156
автомобиль      3903  403 10.325


**Вывод** 
1. Как видим, у цели на **недвижимость** самый низкий показатель должников, скорее всего это обосновано тем, что кредит на недвижимость с самым маленьким процентом по кредиту, и он в основном на долгий срок.
2. У остальных целей процент по кредиту выше и выдается на более маленький срок, что влечет за собой большую вероятность оказаться в долгах. 

## Шаг 4. Общий вывод

1. Выявили зависимости какая доля должников от определеных показателей и сделали соответсвующие выводы. На основе данных зависимостей можно сделать некоторые выводы, но для полной картины не достает еще немало факторов.(Например: в вопросе определение доли должников от количества детей. Очень много различных поддержек для многодетных семей, но не все о них знают) 
2. На основе произведенной нами предобработки данных, можно сказать, что в датасет попадают неккоректные данные. Это может быть связанно с тем, что система заполнения заявки на кредит не доработана:
<ul>
     <li>не реализована валидация полей формы </li>
     <li>лучше использовать маски для полей</li>
     <li>также желательно использовать автоматические счетчики для подсчета стажа работы(вводишь дату дд.мм.гг начала работы и вводишь дату дд.мм.гг окончания работы(или по сей день))</li>
     <li>отпределиться с типами полей</li>
     <li>для борьбы с дубликатами, если в системе присутсвует кнопка 'Оформить заявку' сделать проверку на отправку.</li>
</ul>






## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.